In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import joblib
import tabulate as tb
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras import Sequential, layers, optimizers, losses
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os, random, numpy as np, tensorflow as tf
from model import FinancialLSTMModel
from tensorflow.keras.layers import GlobalAveragePooling1D
from custom_attention import CustomAttention

SEED = 42
os.environ["PYTHONHASHSEED"]=str(SEED)
os.environ["TF_DETERMINISTIC_OPS"]="1"
os.environ["TF_CUDNN_DETERMINISTIC"]="1"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2025-12-01 17:08:16.148138: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-01 17:08:16.223109: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 17:08:18.038091: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
CSV_PATH = './../data/AAPL_1h.csv'
DATE_COL = 'Datetime'

SEQ_LENGTH = 90
BATCH_SIZE = 32
LEARNING_RATE = 0.000001
EPOCHS = 100
TEST_RATIO = 0.2
VAL_SPLIT = 0.1

REPS = 3

EXCLUDE_COLUMNS = ['Datetime', 'returns', 'direction'] 
FEATURES = [
    # Price and Volume Data (Best with Standard Scaling)
    ('Close', 'standard'),
    ('High', 'standard'),
    ('Low', 'standard'),
    ('Open', 'standard'),
    ('Volume', 'standard'),
    ('log_returns', 'standard'),

    # Momentum, Trend & Volatility Indicators (Best with Standard Scaling)
    ('macd', 'standard'),
    ('roc', 'standard'),
    ('adx', 'standard'),
    ('di_plus', 'standard'),
    ('di_minus', 'standard'),
    ('atr_14', 'standard'),
    ('atr_20', 'standard'),
    ('atr_5', 'standard'),
    ('volume_zscore_50', 'standard'),
    
    # Moving Averages & Bands (Best with Standard Scaling)
    ('ema_10', 'standard'),
    ('ema_20', 'standard'),
    ('ema_50', 'standard'),
    ('ema_100', 'standard'),
    ('ema_200', 'standard'),
    ('bb_lower_20', 'standard'),
    ('bb_middle_20', 'standard'),
    ('bb_upper_20', 'standard'),
    
    # Indicators that are Bounded or Ratios (Best with MinMax Scaling)
    ('rsi_14', 'minmax'),
    ('rsi_28', 'minmax'),
    ('rsi_50', 'minmax'),
    ('rsi_7', 'minmax'),
    ('stoch_k', 'minmax'),
    ('stoch_d', 'minmax'),
    ('close_pos', 'minmax'),         # Position of close within the bar
    ('body_range_ratio', 'minmax'),  # Candle body size vs. total range
    ('bb_width_20', 'minmax'),       # Bounded ratio: BB width as a fraction
    
    # Others
    ('obv', 'standard'),
    ('rolling_max_20', 'standard'),
    ('rolling_min_20', 'standard'),
    ('price_from_20d_high', 'standard'),
]

TARGET = 'direction'

def build_hidden_layers1():
    return [
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.LSTM(128, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.LSTM(64, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.4),
        
        CustomAttention(name='attention_layer'),
        
        GlobalAveragePooling1D(name='context_vector_aggregation'),
    ]


In [3]:
res = []

for r in range(REPS):
    print(f"--- REPETITION {r+1}/{REPS} ---")
    model = FinancialLSTMModel(
        csv_path=CSV_PATH,
        features_scales=FEATURES,
        target_col="direction",
        datetime_col="Datetime",

        seq_length=SEQ_LENGTH,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        epochs=EPOCHS,
        test_ratio=TEST_RATIO,
        val_split=VAL_SPLIT,
        weight_adj_factors=[1.5,0.7,1.2],
        under_sample_imbalanced=True
    )

    model.prepare_data()
    model.build_model(build_hidden_layers1())
    model.train()
    ev = model.evaluate()
    res.append(ev)
    
    print(f">> results: {ev}")

--- REPETITION 1/3 ---


2025-12-01 17:08:19.741322: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100


2025-12-01 17:08:20.095355: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_16}}


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - auc_roc: 0.4979 - balanced_accuracy: 0.3527 - loss: 1.2465

...

>> results: {'accuracy': 0.16927899686520376, 'f1_score': 0.1341421683943351, 'balanced_accuracy': 0.3536391042204996, 'precision': np.float32(0.8857143), 'recall': np.float32(0.5477032), 'confusion_matrix': array([[ 32,   6,  34],
       [219,  28, 233],
       [ 37,   1,  48]])}
--- REPETITION 3/3 ---
Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - auc_roc: 0.5014 - balanced_accuracy: 0.3261 - loss: 1.2472

2025-12-01 17:15:05.791378: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - auc_roc: 0.4994 - balanced_accuracy: 0.3303 - loss: 1.2452 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3333 - val_loss: 1.0972 - learning_rate: 1.0000e-06
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - auc_roc: 0.5003 - balanced_accuracy: 0.3673 - loss: 1.2450 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3333 - val_loss: 1.0971 - learning_rate: 1.0000e-06
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - auc_roc: 0.4994 - balanced_accuracy: 0.3419 - loss: 1.2451 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3333 - val_loss: 1.0970 - learning_rate: 1.0000e-06
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - auc_roc: 0.4912 - balanced_accuracy: 0.3527 - loss: 1.2451 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3207 - val_loss: 1.0970 - learning_rate: 1.0000e-06
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - auc_roc: 0.5008 - balanced_accuracy: 0.3613 - loss: 1.2470
Epoch 5: ReduceLROnPlateau reducing learnin

2025-12-01 17:17:16.347734: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


>> results: {'accuracy': 0.20532915360501566, 'f1_score': 0.196026841342712, 'balanced_accuracy': 0.3481750645994832, 'precision': np.float32(0.8938053), 'recall': np.float32(0.89222616), 'confusion_matrix': array([[ 12,   5,  55],
       [ 45,  53, 382],
       [ 16,   4,  66]])}


In [4]:
df = pd.DataFrame(res)

print("\n=== SUMMARY ===")
print(tb.tabulate(df, headers='keys', tablefmt='pretty', showindex="always"))


=== SUMMARY ===
+---+---------------------+---------------------+--------------------+--------------------+---------------------+------------------+
|   |      accuracy       |      f1_score       | balanced_accuracy  |     precision      |       recall        | confusion_matrix |
+---+---------------------+---------------------+--------------------+--------------------+---------------------+------------------+
| 0 | 0.13793103448275862 | 0.07926919532839061 | 0.3326388888888889 | 0.8846153616905212 | 0.04063604399561882 |  [[ 69   3   0]  |
|   |                     |                     |                    |                    |                     |   [461  19   0]  |
|   |                     |                     |                    |                    |                     |  [ 82   4   0]]  |
| 1 | 0.16927899686520376 | 0.1341421683943351  | 0.3536391042204996 | 0.8857142925262451 |  0.547703206539154  |  [[ 32   6  34]  |
|   |                     |                     |   